In [21]:
import torch
import torch.nn as nn
import sys
import nltk
import torch.nn.functional as F
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
sys.path.insert(0, '..')
import pickle
from torch.autograd import grad as torch_grad



word_embedding_dim = 50
z_size = 100
output_shape = (1, 105, 8)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [23]:
class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim, output_shape):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.word_embedding_dim = word_embedding_dim

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105*8)  # Increase the size for more complexity
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105*8)  # Increase the size for more complexity
        self.conv1 = nn.Conv2d(2, 128, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(128)
        self.relu = nn.LeakyReLU(0.2)

        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)
        self.tanh = nn.Tanh()

    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105,8)  # Adjust the size to match conv1

        # Process word embedding
        word_embedding = self.fc_word_embedding(word_embedding.to(device))
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 8)  # Adjust the size to match conv1

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)

        z = self.conv2(z)
        z = self.bn2(z)
        z = self.relu(z)

        z = self.conv3(z)
        z = self.tanh(z)

        return z

In [24]:
# Assuming gen_model is the generator model you used during training
gen_model = Generator(z_size, word_embedding_dim, output_shape)  # Replace with your actual generator model class
checkpoint = torch.load(r"I:\Science\CIS-YASHMOSH\niallmcguire\WGAN_Text_2.0\Textual_WGAN_GP_checkpoint_epoch_100.pt",
                        map_location=torch.device('cpu'))  # Load the final model checkpoint


In [25]:
# Load the model's state_dict onto the CPU
gen_model.load_state_dict(checkpoint['gen_model_state_dict'])

# Set the model to evaluation mode
gen_model.eval()

Generator(
  (fc_noise): Linear(in_features=100, out_features=840, bias=True)
  (fc_word_embedding): Linear(in_features=50, out_features=840, bias=True)
  (conv1): Conv2d(2, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): LeakyReLU(negative_slope=0.2)
  (conv2): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (tanh): Tanh()
)

In [26]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

In [27]:
def generate_samples(g_model, input_z, input_t):
    # Create random noise as input to the generator
    # Generate samples using the generator model
    with torch.no_grad():
        g_output = g_model(input_z, input_t)

    return g_output.cpu().numpy()

In [28]:
import pickle

# To load the lists from the file:
with open(r"C:\Users\gxb18167\PycharmProjects\EEG-To-Text\SIGIR_Development\EEG-GAN\EEG_Text_Pairs.pkl", 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [29]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in EEG_word_level_labels:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

def create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels):
    #EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.std(EEG_word_level_embeddings)

    # Assuming EEG_synthetic is the generated synthetic EEG data
    #EEG_synthetic_denormalized = (EEG_synthetic * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)


    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.max(np.abs(EEG_word_level_embeddings))


    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float)
    float_tensor = float_tensor.unsqueeze(1)

    #print(EEG_word_level_embeddings_normalize)
    # Calculate mean and standard deviation
    print(torch.isnan(float_tensor).any())

    train_data = []
    for i in range(len(float_tensor)):
       train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader

In [31]:
Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)
trainloader = create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels)

Number of word embeddings: 2397
tensor(False)


In [41]:
i1, l1 = next(iter(trainloader))
word_embedding = l1
input_z = create_noise(64, 100, "uniform")

In [42]:
g_output = generate_samples(gen_model, input_z, word_embedding)

In [54]:
g_output[3][0]

array([[-1.47778662e-02, -1.53538510e-02, -6.55647414e-03,
         6.21295534e-03,  1.78094376e-02,  8.31247121e-03,
        -1.25180660e-02, -1.19764963e-02],
       [-1.16977422e-02,  5.69443405e-03,  6.74839946e-04,
        -2.66785212e-02, -1.43437861e-02, -1.07338764e-02,
        -1.29393330e-02, -1.39826369e-02],
       [-1.66216195e-02, -9.48293507e-03,  7.70511199e-03,
        -7.64695648e-03, -7.32490001e-03, -9.24900826e-03,
        -1.63759512e-03,  3.01327673e-03],
       [ 1.35217523e-02,  2.34491704e-03,  2.67614727e-03,
         1.21990161e-03,  5.80506213e-03,  2.56959628e-02,
        -9.21878964e-03,  9.14799049e-03],
       [ 1.09464815e-03,  2.42694300e-02,  2.88138296e-02,
        -6.55050902e-03,  3.45416483e-03,  3.92671190e-02,
         2.55509950e-02,  3.23151350e-02],
       [ 2.68846061e-02,  3.07700541e-02,  2.39659082e-02,
         2.19669621e-02,  2.91935075e-02, -6.08445937e-03,
         1.00673167e-02,  3.24129425e-02],
       [ 2.58742515e-02,  2.183920

In [59]:
# Assuming EEG_synthetic is the generated synthetic EEG data
EEG_synthetic_denormalized = (g_output * np.max(np.abs(EEG_word_level_embeddings))) + np.mean(EEG_word_level_embeddings)


In [64]:
EEG_synthetic_denormalized[0][0]

array([[ 0.9718769 ,  1.1493713 ,  0.9146215 ,  1.5178993 ,  1.1276412 ,
         0.7135271 ,  0.75122   ,  1.0420413 ],
       [ 1.0793734 ,  1.4624445 ,  1.2964745 ,  0.8512777 ,  1.3734119 ,
         1.4755005 ,  2.6217463 ,  2.8937843 ],
       [ 0.9546084 ,  1.0475134 ,  2.247272  ,  0.8192687 ,  2.4663382 ,
         1.8770273 ,  2.1247907 ,  0.8498886 ],
       [ 0.7802946 ,  1.4529914 ,  1.5281878 ,  2.0040357 ,  1.5395465 ,
         1.6145993 ,  1.0308857 ,  0.9423761 ],
       [ 1.5558562 ,  1.9556074 ,  1.9982523 ,  1.1643701 ,  1.4083564 ,
         1.6029782 ,  1.5406584 ,  2.4215708 ],
       [ 2.3024714 ,  2.705129  ,  2.3618584 ,  2.4159517 ,  1.5727917 ,
         1.019681  ,  2.2716465 ,  3.5958128 ],
       [ 2.9563782 ,  2.3291104 ,  2.311779  ,  2.593199  ,  2.7789311 ,
         3.4627388 ,  4.163514  ,  3.9821382 ],
       [ 3.8991277 ,  3.2333212 ,  2.8831315 ,  2.8211827 ,  4.8633823 ,
         4.2833366 ,  3.684194  ,  3.3778937 ],
       [ 1.7499548 ,  1.7650129 